In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

import pickle
import matplotlib.pyplot as plt
import numpy as np

In [11]:
# Change the MODEL_PATH to your local model path
MODEL_PATH = 'aamlm/models/resnet34_tinyimagenet.pth'

target_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Change num_classes to 200 when you use the Tiny ImageNet dataset
target_model = models.resnet34(num_classes=200).to(target_device)

state_dict = torch.load(MODEL_PATH, map_location=target_device)
target_model.load_state_dict(state_dict['net'])

# Test accuracy
acc = state_dict['acc']

# Training epoch (start from 0)
epoch = state_dict['epoch']

#Accuracy = 72.24666666666667

In [12]:
# Change the DATA_PATH to your local pickle file path
DATA_PATH = 'aamlm/pickle/cifar10/mobilenetv2/shadow.p'

data_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=False, num_workers=2)


In [13]:
generator1 = torch.Generator().manual_seed(42)
train_set_size = int(len(dataset) * 0.8)
valid_set_size = len(dataset) - train_set_size
data = torch.utils.data.random_split(dataset, [train_set_size, valid_set_size], generator=generator1)


#With the trainloader variable, you can iterate over the training set in your training loop,
#performing operations on each batch of data.
trainloader = torch.utils.data.DataLoader(data[0], batch_size=128, shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(data[1], batch_size=128, shuffle=False, num_workers=2)

In [14]:
shadow_model_train_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

shadow_model = models.mobilenet_v2(num_classes=10).to(shadow_model_train_device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(shadow_model.parameters(), lr=0.001)

shadow_model.to(shadow_model_train_device)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [15]:
# LOADING THE SAVED SHADOW MODEL

# Change the MODEL_PATH to your local model path
SHADOW_MODEL_PATH = 'my_models/01_07/cifar_mobilenetv2-01_07-2.pth'

shadow_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Change num_classes to 200 when you use the Tiny ImageNet dataset
shadow_model = models.mobilenet_v2(num_classes=10).to(shadow_device)

shadow_model.load_state_dict(torch.load('my_models/01_07/cifar_mobilenetv2-01_07-2.pth',map_location=shadow_device))
shadow_model.eval()  # Set the shadow model to evaluation mode

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [16]:
### GETTING TRAIN RESULTS ON LOADED MODEL

correct = 0
total = 0

shadow_outputs_train = []

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for i, (images, labels) in enumerate(trainloader):
        images = images.to(shadow_model_train_device)
        labels = labels.to(shadow_model_train_device)

        # calculate outputs by running images through the network
        outputs = shadow_model(images)

        # Append the outputs to the shadow_outputs list
        shadow_outputs_train.append(outputs.detach().cpu())

        # the class with the highest energy is what we choose as prediction
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of network on test images: {100 * correct // total}%')
shadow_outputs_train = torch.cat(shadow_outputs_train)

Accuracy of network on test images: 95%


In [17]:
# TESTING THE MODEL ON TESTLOADER
correct = 0
total = 0

shadow_outputs_test = []

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for i, (images, labels) in enumerate(testloader):
        images = images.to(shadow_model_train_device)
        labels = labels.to(shadow_model_train_device)

        # calculate outputs by running images through the network
        outputs = shadow_model(images)

        # Append the outputs to the shadow_outputs list
        shadow_outputs_test.append(outputs.detach().cpu())

        # the class with the highest energy is what we choose as prediction
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of network on test images: {100 * correct // total}%')
shadow_outputs_test = torch.cat(shadow_outputs_test)

#Accuracy of shadow model comes out to be: 53%

Accuracy of network on test images: 57%


In [18]:
members = []
non_members = []
for _, (images, labels) in enumerate(trainloader):
    images = images.to(shadow_model_train_device)
    output = shadow_model(images)
    prediction = F.softmax(output, dim=1)
    top_p,_ = prediction.topk(3, dim=1)
    top_p = top_p.cpu().detach().numpy()
    members.append(top_p)

for _, (images, labels) in enumerate(testloader):
    images = images.to(shadow_model_train_device)
    output = shadow_model(images)
    prediction = F.softmax(output, dim=1)
    top_p,_ = prediction.topk(3, dim=1)
    top_p = top_p.cpu().detach().numpy()
    non_members.append(top_p)

members_features = np.concatenate(members)
member_labels = np.ones(len(members_features))
members_data = np.concatenate((members_features, member_labels.reshape(-1,1)), axis=1)

non_members_features = np.concatenate(non_members)
non_member_labels = np.zeros(len(non_members_features))
non_members_data = np.concatenate((non_members_features, non_member_labels.reshape(-1,1)), axis=1)

In [19]:
combined_data = np.concatenate((members_data, non_members_data), axis=0)
with open('my_models/01_07/shadow_posterior_in_out.p', 'wb') as file:
    pickle.dump(combined_data, file)

In [20]:
# Getting data for Attack model and making a loader out of it

from torch.utils.data import DataLoader, TensorDataset
ATTACK_TRAIN_DATA_PATH = "my_models/01_07/shadow_posterior_in_out.p"

with open(ATTACK_TRAIN_DATA_PATH, "rb") as f:
    attack_shadow_training_data = pickle.load(f)

features = attack_shadow_training_data[:, :-1]
labels = attack_shadow_training_data[:, -1]

# Convert the data to PyTorch tensors
features_tensor = torch.tensor(features, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

# Create a TensorDataset for the combined data
attack_dataset = TensorDataset(features_tensor, labels_tensor)
attack_loader = DataLoader(attack_dataset, batch_size=64, shuffle=False)

In [21]:
# Define the attack model architecture
class AttackModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(AttackModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [22]:
# Building the Attack Model

# Set the dimensions for input, hidden, and output layers of the attack model
input_size = features_tensor.size(1)
hidden_size = 64  # Choose the appropriate size for your problem
num_classes = 2  # Binary classification (shadow or non-shadow)

# Create an instance of the attack model
attack_model = AttackModel(input_size, hidden_size, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(attack_model.parameters(), lr=0.001)

# Set the device for computation
attack_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model and data to the appropriate device
attack_model.to(attack_device)
features_tensor = features_tensor.to(attack_device)
labels_tensor = labels_tensor.to(attack_device)

In [23]:
# Load the attack model trained on shadow.p posterior

# Change the MODEL_PATH to your local model path
ATTACK_MODEL_PATH = 'my_models/01_07/attack_model-1.pth'

attack_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

attack_model.load_state_dict(torch.load('my_models/01_07/attack_model-1.pth',map_location=attack_device))
attack_model.eval() 

AttackModel(
  (fc1): Linear(in_features=3, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)

In [29]:
EVAL_DATA_PATH = 'aamlm/pickle/tinyimagenet/resnet34/eval.p'

eval_device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(EVAL_DATA_PATH, "rb") as f:
    eval_dataset = pickle.load(f)

eval_loader = DataLoader(eval_dataset, batch_size=64, shuffle=False)

In [30]:
# Getting the posteriors and ground_truth from eval.p
# Posteriors are sent to attack model
# Ground_truth is used in finding accuracy with attack model

eval_posteriors = []
eval_groundTruth = []

with torch.no_grad():
    for batch_idx, (images, labels, ismember) in enumerate(eval_loader):
        images = images.to(eval_device)
        labels = labels.to(eval_device)
        ismember = ismember.to(eval_device)

        # Calculate posteriors using the pretrained target model
        logits = target_model(images)
        probs = F.softmax(logits, dim=1)

        # Get the top-k posteriors
        topk_probs, _ = probs.topk(3, dim=1)

        # Append the posteriors and ismember to the lists
        eval_posteriors.append(topk_probs.cpu())
        eval_groundTruth.append(ismember.cpu())

eval_posteriors = torch.cat(eval_posteriors)
eval_groundTruth = torch.cat(eval_groundTruth)

In [31]:
eval_data_combined = np.concatenate((eval_posteriors, eval_groundTruth.reshape(-1,1)), axis=1)
print(eval_data_combined.shape)

with open('my_models/01_07/eval_data_combined', 'wb') as file:
    pickle.dump(eval_data_combined, file)

# here we save the eval dataset which just has

(200, 4)


In [32]:
# Loading the saved eval combined dataset and getting the dataloader from that data

with open('my_models/01_07/eval_data_combined', "rb") as file:
    eval_loaded_data = pickle.load(file)

eval_features = eval_loaded_data[:, :-1]
eval_labels = eval_loaded_data[:, -1]

# Convert the data to PyTorch tensors
features_tensor = torch.tensor(eval_features, dtype=torch.float32)
labels_tensor = torch.tensor(eval_labels, dtype=torch.long)

# Create a TensorDataset for the combined data
evalattack_dataset = TensorDataset(features_tensor, labels_tensor)
evalattack_loader = DataLoader(evalattack_dataset, batch_size=64, shuffle=False)

In [33]:
# Making predictions on attack model for eval dataset with only images, groundTruth

attack_model.eval()

predictions = []

correct = 0
total = 0

for batch_idx, (posteriors, ismember) in enumerate(evalattack_loader):

    posteriors = posteriors.to(eval_device)
    ismember = ismember.to(eval_device)

    outputs = attack_model(posteriors)
    _, predicted = torch.max(outputs.data, 1)
    predictions.append(predicted.cpu().numpy())

    total += ismember.size(0)
    correct += (predicted == ismember).sum().item()

predictions = np.concatenate(predictions)

# Calculate the accuracy
accuracy = 100 * correct / total
print(f"Attack Model Accuracy: {accuracy:.2f}%")

np.save("attack_predictions.npy", predictions)

Attack Model Accuracy: 96.50%


In [56]:
TEST_DATA_PATH = 'aamlm/pickle/tinyimagenet/resnet34/test.p'

test_device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(TEST_DATA_PATH, "rb") as f:
    test_dataset = pickle.load(f)

test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False,num_workers=2)

In [57]:
len(test_loader)

313

In [58]:
# Getting the posteriors and ground_truth from test.p
# Posteriors are sent to attack model
# Ground_truth is used in finding accuracy with attack model

test_posteriors = []
#test_groundTruth = []

target_model.eval()

with torch.no_grad():
    for batch_idx, (images, labels) in enumerate(test_loader):
        images = images.to(test_device)
        labels = labels.to(test_device)
        

        # Calculate posteriors using the pretrained target model
        logits = target_model(images)
        probs = F.softmax(logits, dim=1)

        # Get the top-k posteriors
        topk_probs, _ = probs.topk(3, dim=1)

        # Append the posteriors and ismember to the lists
        test_posteriors.append(topk_probs.cpu())
        #test_groundTruth.append(ismember.cpu())

test_posteriors = torch.cat(test_posteriors)
#test_groundTruth = torch.cat(test_groundTruth
torch.save(test_posteriors, 'my_models/test_posteriors_data.p')

In [59]:
print(test_posteriors.shape)

torch.Size([20000, 3])


In [60]:
test_labels = [item[-1] for item in test_dataset]

print(test_labels)

test_ground_truth_array = np.array(test_labels, dtype=int)

[5, 166, 191, 81, 181, 146, 2, 155, 33, 57, 90, 58, 173, 100, 178, 50, 11, 115, 58, 170, 179, 159, 59, 13, 144, 15, 142, 51, 135, 36, 171, 3, 71, 164, 112, 175, 143, 111, 24, 126, 87, 99, 179, 45, 191, 21, 70, 11, 150, 18, 138, 147, 23, 45, 88, 153, 114, 96, 196, 162, 57, 161, 196, 44, 172, 120, 117, 167, 123, 194, 75, 90, 199, 199, 120, 113, 75, 118, 115, 1, 140, 152, 77, 7, 142, 73, 170, 144, 48, 44, 143, 105, 71, 80, 163, 15, 14, 80, 150, 177, 151, 99, 52, 99, 198, 179, 139, 34, 167, 140, 54, 7, 140, 143, 184, 195, 188, 194, 26, 163, 156, 46, 183, 146, 121, 44, 15, 5, 141, 40, 114, 116, 75, 191, 131, 169, 177, 176, 152, 170, 54, 39, 92, 35, 133, 5, 63, 149, 164, 194, 178, 15, 120, 70, 82, 36, 145, 4, 133, 146, 160, 10, 157, 57, 161, 155, 79, 156, 65, 62, 83, 153, 56, 75, 153, 142, 49, 58, 113, 93, 159, 61, 151, 16, 180, 164, 95, 198, 117, 172, 50, 184, 8, 121, 168, 55, 109, 147, 23, 174, 192, 119, 183, 138, 67, 93, 2, 157, 106, 116, 3, 196, 133, 18, 88, 72, 164, 165, 38, 32, 109, 92

In [61]:
test_posteriors_array = test_posteriors.numpy()
final_test_data_combined = np.concatenate((test_posteriors, test_ground_truth_array.reshape(-1,1)), axis=1)
with open('my_models/test_posteriors_data.p', 'wb') as file:
    pickle.dump(final_test_data_combined, file)

print(final_test_data_combined)
print(final_test_data_combined.shape)

[[7.86161900e-01 7.89848343e-02 5.76777495e-02 5.00000000e+00]
 [9.56422627e-01 4.35757376e-02 1.32207515e-06 1.66000000e+02]
 [9.99999523e-01 1.87527874e-07 1.78977544e-07 1.91000000e+02]
 ...
 [9.99999046e-01 1.00749696e-06 5.21346095e-08 6.70000000e+01]
 [1.00000000e+00 7.67023334e-09 3.35193623e-10 2.50000000e+01]
 [1.00000000e+00 1.15442695e-38 4.40007718e-43 6.70000000e+01]]
(20000, 4)


In [63]:
with open('my_models/test_posteriors_data.p', "rb") as file:
    final_test_data = pickle.load(file)

test_features = final_test_data[:, :-1]
test_labels = final_test_data[:, -1]
print(test_features.shape)
print(test_labels.shape)
# Convert the data to PyTorch tensors
test_features_tensor = torch.tensor(test_features, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)

# Create a TensorDataset for the combined data
attack_final_test_tensor = TensorDataset(test_features_tensor, test_labels_tensor)

attack_final_test_loader = DataLoader(attack_final_test_tensor, batch_size=64, shuffle=False)

(20000, 3)
(20000,)


In [68]:
attack_model.eval()

predictions = []

with torch.no_grad():
    for batch,(images,labels) in enumerate(attack_final_test_loader):
        images = images.to(test_device)

        outputs = attack_model(images)
        predicted_probs = torch.softmax(outputs, dim=1)
        highest_prob = torch.argmax(predicted_probs, dim=1)

        predictions.extend(highest_prob.cpu().numpy())

#np.array(predictions)

# Save predictions to .npy file
np.save('my_models/task2_resnet34_tinyimagenet.npy', predictions)
 

In [65]:
print(len(predictions))

20000


In [67]:
unique_elements = list(set(predictions))
print(unique_elements)

[0, 1]


In [71]:
count_0 = 0 
count_1 = 0

for i in predictions:
    if i==0:
        count_0+=1
    else:
        count_1+=1
print("Member Count: " + str(count_1))
print("Non-Member Count: " + str(count_0))

Member Count: 15950
Non-Member Count: 4050
